In [7]:
from typing import Any

from networkx.algorithms.centrality import group_in_degree_centrality


def weighted_sum(x,weight):
    return (weight*x).sum(axis=-1)

In [8]:
import triton
import triton.language as tl

In [9]:
@triton.jit
def weighted_sum_fwd(
        x_ptr,weight_ptr, #这是输入的指针
        output_ptr,       #这是输出的指针
        x_stride_row,x_stride_dim, #步长信息：张量各个维度的元素间隔
        weight_stride_dim, #通常为1
        output_stride_row, #通常为1
        ROWS,D,
        ROWS_TILE_SIZE:tl.constexpr,D_TILE_SIZE:tl.constexpr,  #区块形状在编译时是定常量
):
    #Triton 使用 SPMD (Single Program, Multiple Data) 模型。启动这个核函数时，GPU会同时启动很多个这个函数的“实例”，也就是诸线程块。
    #每个实例计算X的若干行加权和
    #tl.program_id获取当前线程块ID
    row_tile_idx=tl.program_id(0)

    #块指针给我们一个操作ND内存区域的方法，并且会随进程移动
    #块指针必须知道：张量首元素指针，张量整体的形状（防止越界），各维度的步长（来正确使用内存布局），起始块的ND坐标（偏移量），单次加载/存储的区块形状，内存维度的顺序(按步长降序，用于H100优化）
    x_block_ptr=tl.make_block_ptr(
        x_ptr,#张量首元素指针
        shape=(ROWS,D,), #整体形状
        strides=(x_stride_row,x_stride_dim),#各个维度的步长
        offsets=(row_tile_idx*ROWS_TILE_SIZE,0), #计算存储实际偏移位置
        block_shape=(ROWS_TILE_SIZE,D_TILE_SIZE), #单次加载的区块形状
        order=(1,0),  #维度顺序
    )

    weight_block_ptr=tl.make_block_ptr(
        weight_ptr,
        shape=(D,),
        strides=(weight_stride_dim,),
        offsets=(0,),  #因为我门只有一行
        block_shape=(D_TILE_SIZE,),
        order=(0,),
    )

    output_block_ptr=tl.make_block_ptr(
        output_ptr,
        shape=(ROWS,),
        strides=(output_stride_row,),
        offsets=(row_tile_idx*ROWS_TILE_SIZE,),
        block_shape=(ROWS_TILE_SIZE,),
        order=(0,),
    )
    #初始化输出缓冲区
    output=tl.zeros((ROWS_TILE_SIZE,),dtype=tl.float32)  #计算仅在这里进行，所有计算进行完成了才把最终结果一次性写回output_ptr

    for i in range(tl.cdiv(D,D_TILE_SIZE)):  #cdiv(D,D_TILE_SIZE)=up[D/D_TILE_SIZE] 相当于一个ceiling_div,保证块内所有元素取完了
        row=tl.load(x_block_ptr,boundary_check=(0,1),padding_option="zero")   #使用创建的x块指针，从全局内存加载到一个小片的数据到寄存器中boundary_check表示两个维度都要进行安全检查，如果越界，我们用0填充
        weight=tl.load(weight_block_ptr,boundary_check=(0,),padding_option="zero") #只有一行小片

        output+=tl.sum(row*weight[None,:],axis=1)      #注意weight的形状是(D_TILE_SIZE,)利用weight[None,:]变成了(1,D_TILE_SIZE)，然后再作元素级乘法，然后再沿着D维度求和形成(R_TILE_SIZE,)张量

        x_block_ptr=x_block_ptr.advance((0,D_TILE_SIZE))             #x的行不动，列方向上前进D_TILE_SIZE
        weight_block_ptr=weight_block_ptr.advance((D_TILE_SIZE,))    #weight也是
    #列的遍历是通过上述for循环实现的，而行的遍历在一开始的row_tile_idx就分出了多个线程来并行处理
    tl.store(output_block_ptr,output,boundary_check=(0,))


In [10]:
import torch
from einops import rearrange
#把刚才的内核函数封装成Pytorch自动求导的函数

class WeightedSumFunc(torch.autograd.Function):
    #缓存x何weight用于反向传播
    #该阶段我们只会收到输出张量的梯度
    #但是需要计算x和weight的梯度
    @staticmethod
    def forward(ctx,x,weight):                                 #ctx作用是从forward方法向backward传递信息，可能用到的值会保存到ctx中
        D,output_dims=x.shape[-1],x.shape[:-1]   #解析输入的x，x的最后一个维度给D，其余的维度给到output_dims

        input_shape=x.shape                      #保存x原始形状到input_shape，我们会把输出变回这个形状
        x=rearrange(x,"... d -> (...) d")        #对x作维度变换，把前面的维度全部合并

        ctx.save_for_backward(x,weight)          #保存变维后的x和weight，因为计算梯度还需要它们

        assert len(weight.shape)==1 and weight.shape[0]==D,  "维度不匹配"                    #确保weight仅有一个维度，且该维度和D相同
        assert x.is_cuda and weight.is_cuda, "x或weight不在cuda"                            #确保张量在gpu上
        assert x.is_contiguous(),   "x未连续存储"                                            #确保张量连续存储
        #下面调整并行策略
        ctx.D_TILE_SIZE=triton.next_power_of_2(D)//16  #triton在处理大小为2的幂的数据块的时候性能最好;我们用next_power_of向上取2的幂次，约16次嵌入维度的循环 ##问这里是否可以实现2的四舍五入？
        ctx.ROWS_TILE_SIZE=16                          #每个线程同时处理16个批次（行）
        ctx.input_shape=input_shape                    #把初始的形状传递到ctx
        #初始一个空结果张量，元素不一定是0！
        y=torch.empty(output_dims,dtype=x.dtype,device=x.device) #在gpu上为输出张量分配内存,使用empty更快，形状是x初始的前面的维度
        #在ID网格中启动n个实例的内核
        n_rows=y.numel()                #获取y的元素总数，因为当前x已经展平了，y.numel()其实就是当前x行数，将它作为ROWS传给triton
        weighted_sum_fwd[(tl.cdiv(n_rows,ctx.ROWS_TILE_SIZE),)](
            x,weight,
            y,
            x.stride(0),x.stride(1),
            weight.stride(0),y.stride(0),
            ROWS=n_rows,D=D,
            ROWS_TILE_SIZE=ctx.ROWS_TILE_SIZE,
            D_TILE_SIZE=ctx.D_TILE_SIZE,
        )#调用triton内核函数，其中[(tl.cdiv(n_rows,ROWS_TILE_SIZE),)]计算出线程数，决定tl.program_id(0)的最大值
        return y.view(input_shape[:-1]) #内核计算出的是一个展平的张量，我们把它的前面维度按初始x的前面维度重塑

    #然后这个backward是写好了weighted_sum_backward()后再回来定义的
    @staticmethod
    def backward(ctx,grad_out):
        x,weight=ctx.saved_tensors                                             #从ctx中取出forward时保存的张量x,weight
        ROWS_TILE_SIZE,D_TILE_SIZE=ctx.ROWS_TILE_SIZE,ctx.D_TILE_SIZE          #取出已定好的ROWS_TILE_SIZE,D_TILE_SIZE
        n_rows,D=x.shape                                                       #n_rows是总行数，D是嵌入维度

        partial_grad_weight=torch.empty((tl.cdiv(n_rows,ROWS_TILE_SIZE),D),device=x.device,dtype=x.dtype)  #partial_grad_weight分配内存，行数就是我们要进行的线程数，列就是D
        grad_x=torch.empty_like(x)

        weighted_sum_backward[(tl.cdiv(n_rows,ROWS_TILE_SIZE),)](
            x,weight,grad_out,grad_x,partial_grad_weight,
            x.stride(0),x.stride(1),
            weight.stride(0),grad_out.stride(0),
            grad_x.stride(0),grad_x.stride(1),
            partial_grad_weight.stride(0),partial_grad_weight.stride(1),
            NUM_ROWS=n_rows,D=D,
            ROWS_TILE_SIZE=ROWS_TILE_SIZE,D_TILE_SIZE=D_TILE_SIZE,
        )
        grad_weight=partial_grad_weight.sum(axis=0)      #我们之前只是算了块的列和，现在是把各个块的列和再次加总
        return grad_x,grad_weight


由于我们定义了自己的内核，因此也需要编写相应的反向传播函数。在前向传播中，我们接收层的输入并计算其输出；而在反向传播中，我们将获得目标函数相对于输出的梯度，并需要计算相对于每个输入的梯度。在我们操作中，输入包括矩阵 $x:\mathbb{R}^{n\times h}$ 和权重向量 $w:\mathbb{R}^{h}$;若简记我们的操作是 $f(x,w)$ 它的值域是 $\mathbb{R}^n$。假设给定 $\nabla_{f(x,w)}L$ ,应用多元的链式法则就有
$$\begin{aligned}
(\nabla_x\mathcal{L})_{ij}&=\sum_{k=1}^n\frac{\partial f(x,w)_k}{\partial x_{ij}}(\nabla_{f(x,w)}\mathcal{L})_k=w_j\cdot(\nabla_{f(x,w)}\mathcal{L})_i\\
(\nabla_w\mathcal{L})_{ij}&=\sum_{i=1}^n\frac{\partial f(x,w)_i}{\partial w_{j}}(\nabla_{f(x,w)}\mathcal{L})_i=\sum_{i=1}^n x_{ij}\cdot(\nabla_{f(x,w)}\mathcal{L})_i
\end{aligned}
$$
这就为计算反向传播提供了简洁的公式，要计算关于 $x$ 的梯度，我们取 $w$ 和 $\nabla_{f(x,w)}$ 的外积；计算关于 $w$ 的梯度，我们需要将输入梯度与对应的输出行相乘。

In [11]:
@triton.jit
def weighted_sum_backward(
        x_ptr,weight_ptr,                        #x,w的指针
        grad_output_ptr,                         #我们的L相对于输出层的梯度指针      （输入）
        grad_x_ptr,partial_grad_weight_ptr,      #待计算的L对于x，weight的梯度的指针（输出）
        stride_xr,stride_xd,                     #X的row步长和X的dim步长
        stride_wd,                               #weight的dim步长
        stride_gr,                               #L对于输出层的梯度的row步长
        stride_gxr,stride_gxd,                   #L对于x的梯度的row步长，L对于x的梯度dim步长
        stride_gwb,stride_gwd,                   #L对于w的梯度的batch步长，L对于w的梯度的dim步长
        NUM_ROWS,D,                              #处理的X行数，维度
        ROWS_TILE_SIZE:tl.constexpr,D_TILE_SIZE:tl.constexpr,  #小片的行数和维度
):
    row_tile_idx=tl.program_id(0)        #获取当前线程块的id
    n_rows_tiles=tl.num_programs(0)      #获取当前的总块数
    #下面是L对输出层的块指针
    grad_output_block_ptr=tl.make_block_ptr(
        grad_output_ptr,                                   #指向grad_output张量的开头
        shape=(NUM_ROWS,),strides=(stride_gr,),            #整个张量的逻辑形状是(NUM_ROWS)，因为我们会处理NUM_ROWS行，然后每行有个输出，(stride_gr) 是内存步长，每个线程每次处理stride_gr个数据
        offsets=(row_tile_idx*ROWS_TILE_SIZE,),            #为当前线程块设置起始偏移，是当前小片的id再去乘每片的rows长，这样可以找到当前线程的首指针
        block_shape=(ROWS_TILE_SIZE,),                     #这个线程块的形状就是处理(ROWS_TILE_SIZE)个然后按步长stride_gr向前处理
        order=(0),                                         #定义块内数据内存布局，这里仅有一个维度
    )
    #下面是x的块指针
    x_block_ptr=tl.make_block_ptr(
        x_ptr,
        shape=(NUM_ROWS,D,),strides=(stride_xr,stride_xd),
        offsets=(row_tile_idx*ROWS_TILE_SIZE,0),
        block_shape=(ROWS_TILE_SIZE,D_TILE_SIZE,),
        order=(1,0),
    )
    #下面是weight的块指针
    weight_block_ptr=tl.make_block_ptr(
        weight_ptr,
        shape=(D,),strides=(stride_wd,),
        offsets=(0,),block_shape=(D_TILE_SIZE,),
        order=(0,),
    )
    #下面是对x的梯度块指针
    grad_x_block_ptr=tl.make_block_ptr(
        grad_x_ptr,
        shape=(NUM_ROWS,D,),strides=(stride_gxr,stride_gxd),
        offsets=(row_tile_idx*ROWS_TILE_SIZE,0),
        block_shape=(ROWS_TILE_SIZE,D_TILE_SIZE,),
        order=(1,0)
    )
    #下面是对权重的梯度块指针,这个指针比较特殊。它不指向最终的grad_weight。计算grad_weight需要对X的所有行进行求和，但是我们已经把行分给了不同线程块，所以每个线程块只能计算出基于它负责的那些个(ROWS_TILE_SIZE)行的梯度，这是一个“部分和”
    partial_grad_weight_block_ptr=tl.make_block_ptr(
        partial_grad_weight_ptr,
        shape=(n_rows_tiles,D,),                          #(总块数,D)，每一行用来存储一个线程块计算出的部分和
        strides=(stride_gwb,stride_gwd),                  #步长分别是(stride_gwb,stride_gwd)
        offsets=(row_tile_idx,0),                         #偏移量仅为id
        block_shape=(1,D_TILE_SIZE,),                     #小片形状
        order=(1,0),                                      #优先列排
    )

    for i in range(tl.cdiv(D,D_TILE_SIZE)):
        grad_output=tl.load(grad_output_block_ptr,boundary_check=(0,),padding_option="zero") #从grad_output_block_ptr指针处加载一个数据块
        weight=tl.load(weight_block_ptr,boundary_check=(0,),padding_option="zero")           #从weight_block_ptr指针处加载一个数据块
        grad_x_row=grad_output[:,None]*weight[None,:]                                        #计算详情回到分析式子，grad_output[:,None]把(R_T_S,)->(R_T_S,1),weight[None,:]把(D_T_S,)->(1,D_T_S) 然后自动广播得到(R_T_S,D_T_S)的梯度矩阵,正是grad_x的若干个row分块
        tl.store(grad_x_block_ptr,grad_x_row,boundary_check=(0,1))                           #把计算好的grad_x块写回grad_x_block_ptr的内存，并防止越界

        row=tl.load(x_block_ptr,boundary_check=(0,1),padding_option="zero")                  #从x_block_ptr加载一个数据块
        grad_weight_row=tl.sum(row*grad_output[:,None],axis=0,keep_dims=True)                #计算详情回到分析式子，row是几个x的行小块,(R_T_S,D_T_S),然后我们的grad_output(R_T_S,)->(R_T_S,D_T_S),与row进行元素级乘法，再每列 每列地相加,得到(1,D_T_S)的部分和列
        tl.store(partial_grad_weight_block_ptr,grad_weight_row,boundary_check=(1,))          #保存好grad_weight，但是只用在列维度检查，因为行维度仅仅是1行

        x_block_ptr=x_block_ptr.advance((0,D_TILE_SIZE))                                     #x的块指针在同一行上向前移动
        weight_block_ptr=weight_block_ptr.advance((D_TILE_SIZE,))                            #weight的块指针也移动
        partial_grad_weight_block_ptr=partial_grad_weight_block_ptr.advance((0,D_TILE_SIZE)) #partial_grad_weight_block_ptr也在同一行上向前移动
        grad_x_block_ptr=grad_x_block_ptr.advance((0,D_TILE_SIZE))                           #grad_x_block_ptr也在同一行上向前移动

通过上述的WeightedSumFunc类 实现了功能类似于 torch.nn.functional 中的一个函数操作:

f\_weightedsum=WeightedSumFunc.apply


我们再次回到 $QKV$ 的缩放点积注意力，对于 $Q^{n\times d},K^{n\times d},V^{n\times v}$, $Q$ 是根据n个词的嵌入信息得到的n个d维的查询行向量，$K^T$ 的每一列是一个键，把每个查询向量分别与这n个键做内积得到注意力评分，再把注意力评分通过softmax变换得到这个查询下取到各个键的概率(一行)，用这一行的n个概率分别去乘V的n个行向量然后相加得到某个查询所取值的加权平均：
$$
\begin{aligned}
    S&=\frac{QK^T}{\sqrt{d}}&\text{按行的注意力评分}\\
    P&=softmax (S)&\text{每一行是一个查询向量查询到这n个键的概率} \\
    O&=PV&\text{每一个行向量是一个查询向量最终取值的加权平均}
\end{aligned}
$$

然后考虑反向传播的求导机制，我们利用 $dX$ 来代替 $\frac{\partial \mathcal{L}}{\partial X}$ 这样有以下式子
$$
\begin{aligned}
    dV&=P^T\ dO\\
    dP&=dO\ V^T\\
    dS_i&=dsoftmax(dP_i)=(diag(P_i)-P_iP_i^T)dP_i\\
    dQ&=dSK/\sqrt{d}\\
    dK&=dS^TQ/\sqrt{d}
\end{aligned}
$$

对于 $dV=P^T\ dO$ ，事实上对于 $Y=AX$我们有
$$
\frac{\partial\mathcal{L}}{\partial X_{ab}}=\sum_i\sum_j\frac{\partial\mathcal{L}}{\partial Y_{ij}}\cdot\frac{\partial Y_{ij}}{\partial X_{ab}}
$$
其中
$$
Y_{ij}=\sum_k A_{ik}X_{kj}
$$
容易发现必须有 $j=b$ 的时候，我们才能取到一个 $\frac{\partial Y_{ib}}{\partial X_{ab}}=A_{ia}$，这样最初的那个式子简化成了
$$
\frac{\partial\mathcal{L}}{\partial X_{ab}}=\sum_i\frac{\partial\mathcal{L}}{\partial Y_{ia}} A_{ia}
$$
当然它也是
$$
\frac{\partial\mathcal{L}}{\partial X_{ab}}=\sum_iA^T_{ai}\frac{\partial\mathcal{L}}{\partial Y_{ia}}
$$
这便是
$$
dX=A^T\ dY
$$
带回 $O=PV$ 就是 $dV=P^T\ dO$

对于 $dP=dO\ V^T$，我们首先有
$$
O_{ij}=\sum_k P_{ik}V_{kj}
$$
然后对于损失求导由链式法则
$$
\frac{\partial \mathcal{L}}{\partial P_{ab}}=\sum_i\sum_j\frac{\partial \mathcal{L}}{\partial O_{ij}}\cdot\frac{\partial O_{ij}}{\partial P_{ab}}
$$
发现 $\frac{\partial O_{ij}}{\partial P_{ab}}$ 仅在 $i=a$ 的时候能取到 $V_{aj}$ 的值，于是
$$
\frac{\partial \mathcal{L}}{\partial P_{ab}}=\sum_j \frac{\partial \mathcal{L}}{\partial O_{aj}}\cdot V_{aj}
$$
亦即
$$
\frac{\partial \mathcal{L}}{\partial P_{ab}}=\sum_j \frac{\partial \mathcal{L}}{\partial O_{aj}}\cdot V^T_{ja}
$$
得到了
$$
dP=dO\ V^T
$$

对于 $dS_i=d softmax(dP_i)=(diag(P_i)-P_iP_i^T)dP_i$ ,我们按某一行来分析；由于 $P_i=softmax(S_i)$,对于分量我们就有
$$
P_{ij}=\frac{e^{S_{ij}}}{\sum_k e^{S_{ik}}}
$$
现在来考虑损失对于 $P_{i},S_{i}$ 的导数，对于某个a分量，我们有
$$
\frac{\partial \mathcal{L}}{\partial S_{ia}}=\sum_j\frac{\partial \mathcal{L}}{\partial P_{ij}}\cdot \frac{\partial P_{ij}}{\partial S_{ia}}
$$
当 $j=a$ 的时候，$\frac{\partial P_{ia}}{\partial S_{ia}}=P_{ia}(1-P_{ia})$；当 $j\ne a$ 的时候 $\frac{\partial P_{ij}}{\partial S_{ia}}=-P_{ia}P_{ij}$,这样再带回上式，我们有
$$\begin{aligned}
\frac{\partial \mathcal{L}}{\partial S_{ia}}&=\sum_{j\ne a}\frac{\partial \mathcal{L}}{\partial P_{ij}}\cdot (-P_{ia}P_{ij})+\frac{\partial \mathcal{L}}{\partial P_{ia}}\cdot P_{ia}(1-P_{ia})\\
&=\frac{\partial \mathcal{L}}{\partial P_{ia}}\cdot P_{ia}-\sum_{j}\frac{\partial \mathcal{L}}{\partial P_{ij}}\cdot P_{ia}P_{ij}
\end{aligned}
$$
从这里开始，尽管我们是按行处理的，但是我们把向量看为是列向量，那么其实 $(diag(P_i)-P_iP_i^T)dP_i$ 这个式子也会变为一个列向量，问题在于每个分量是多少？事实上
$$
(diag(P_i)-P_iP_i^T)\left(\begin{aligned}\frac{\partial\mathcal{L}}{\partial P_{i1}}\\
\frac{\partial\mathcal{L}}{\partial P_{i2}}\\
.\\.\\.\\
\frac{\partial\mathcal{L}}{\partial P_{in}}\end{aligned}\right)=diag(P_i)\left(\begin{aligned}\frac{\partial\mathcal{L}}{\partial P_{i1}}\\
\frac{\partial\mathcal{L}}{\partial P_{i2}}\\
.\\.\\.\\
\frac{\partial\mathcal{L}}{\partial P_{in}}\end{aligned}\right)-P_iP_i^T\left(\begin{aligned}\frac{\partial\mathcal{L}}{\partial P_{i1}}\\
\frac{\partial\mathcal{L}}{\partial P_{i2}}\\
.\\.\\.\\
\frac{\partial\mathcal{L}}{\partial P_{in}}\end{aligned}\right)
$$
简单计算发现确实与 $\frac{\partial \mathcal{L}}{\partial P_{ia}}\cdot P_{ia}-\sum_{j}\frac{\partial \mathcal{L}}{\partial P_{ij}}\cdot P_{ia}P_{ij}$ 对上了.

来到 $dQ=dSK/\sqrt{d}$ 没啥好说的，我们直接利用第二个式子的结论：$O=PV\Rightarrow dP=dO V^T$，那么对于 $S=QK^T/\sqrt{d}$ 我们就有
$$
dQ=dS [K^T/\sqrt{d}]^T\Leftrightarrow dQ=dS(K/\sqrt{d})
$$

最后是 $dK=dS^T\ Q/\sqrt{d}$，和上一个一样，我们只是作了个转置 $S^T=KQ^T/\sqrt{d}$，这样就有
$$
dK=dS^T\ Q/\sqrt{d}
$$

传统的 qkv缩放点积注意力的反向传播依赖于前向传播产生的一些超大张量，比如要计算 $dV$ 就需要一个形状 (batch_size,n_heads,seq_len,seq_len) 的注意力分数矩阵 $P$ 该矩阵的尺寸会随着序列长度呈平方增长，这便会产生内存的瓶颈。在标准注意力的前后向传播中我们需要为 $P$ 这样的大张量在 SRAM和GPU的HBM之间进行多次数据传输，造成巨大的IO开销。

FlashAttention的核心目标是通过以下三个技术来避免注意力矩阵的HBM读写：

1.分块计算(Tiling)

2.重计算(Recomputation)

3.算子融合(Operator Fusion)

Tiling:为了避免注意力矩阵在HBM中的读写，我们采用无需全局输入访问的softmax计算。简言之，通过把输入分割为多个区块并多次遍历这些区块，实现增量式的softmax计算.

Recomputation:我们避免在HBM中存储形状为(batch_size,n_heads,seq_len,seq_len) 的大型中间矩阵；取而代之的是在HBM中保存特定的激活检查点，并在反向传播的时候重新计算部分前向过程来获取梯度计算所需的激活值。在FlashAttention-2中额外存储注意力分数的 logsumexp值 $L$，表达式是
$$
L_i=\log \left(\sum_j \exp\boldsymbol(S_{ij})\right)
$$
在最终内核的实现时候，我们会以在线方式计算该值，结果会保持一致。通过分块与重计算的结合，内存IO和峰值使用量不再与 seq_len 相关，从而支持更长序列的处理。

Operator Fusion:最后，我们通过把所有操作集成到单个内核中执行(称作算子/内核融合)，避免注意力矩阵以及其他中间值的重复内存IO。将编写一个统一的Triton前向传播核方法，在有限的HBM与SRAM数据传输下完成所有注意力相关运算。重计算部分实现了算子融合，因此可以避免常规实现中存储所有中间值到HBM的内存开销。

重计算下的反向传播与传统qkv的反向传播有点区别；通过使用 $L$，我们能够高效地执行重计算并完成反向传播的计算。

在执行反向传播过程之前，我们会预先在全局内存中计算中间值 $D=rowsum(\boldsymbol{O}\odot d\boldsymbol{O})$ ，这个值等价于 $rowsum(\boldsymbol{P}\odot d\boldsymbol{P})$ ，这是因为存在以下的恒等式
$$
\boldsymbol{P}d\boldsymbol{P}^T=\boldsymbol{P}(d\boldsymbol{O}\boldsymbol{V}^T)^T=(\boldsymbol{PV}d\boldsymbol{O}^T)=\boldsymbol{O}d\boldsymbol{O}^T
$$
并且对于任意矩阵 $A,B$ 我们总有 $rowsum(\boldsymbol{A}\odot \boldsymbol{B})=diag(\boldsymbol{AB}^T)$。结合 $L,D$ ，反向传播计算可以完全规避 $softmax$ 的运算。完整的公式一览如下：
$$
\begin{aligned}
    S&=QK^T/\sqrt{d}\\
    P_{ij}&=\exp(S_{ij}-L_i)\\
    dV&=P^TdO\\
    dP&=dO\ V^T\\
    dS_{ij}&=P_{ij}\odot(dP_{ij}-D_i)\\
    dQ&=dSK/\sqrt{d}\\
    dK&=dS^TQ/\sqrt{d}
\end{aligned}
$$
其它的部分我们在传统qkv里面已做说明，来看看新增的 $dS_{ij}=P_{ij}\odot(dP_{ij}-D_i)$，它其实是在 $D$ 替换之后的
$$\begin{aligned}
\frac{\partial \mathcal{L}}{\partial S_{ia}}&=\sum_{j\ne a}\frac{\partial \mathcal{L}}{\partial P_{ij}}\cdot (-P_{ia}P_{ij})+\frac{\partial \mathcal{L}}{\partial P_{ia}}\cdot P_{ia}(1-P_{ia})\\
&=\frac{\partial \mathcal{L}}{\partial P_{ia}}\cdot P_{ia}-\sum_{j}\frac{\partial \mathcal{L}}{\partial P_{ij}}\cdot P_{ia}P_{ij}
\end{aligned}
$$
因为 $D_i$ 实际上是 $\sum\limits_{j}P_{ij}\frac{\partial\mathcal{L}}{\partial P_{ij}}$，直接带回就发现是恒等的。

从上述计算可见，我们无需在前向传播阶段把注意力分数 $P$ 存储与HBM，这些中间值可以通过激活张量 $Q,K$ 和归一化因子 $L$ 在公式中重计算获得。

一些细节问题：为了规避HBM的读写，我们会采用分块计算策略--独立计算出张量的每个区块，这要求能对注意力矩阵 $P$ 实现双向分块计算 (同时在查询维度和键维度分块)

但是对 $S$ 矩阵应用 $softmax$ 时候，需要来看整行的元素来计算，这意味着 $P$ 矩阵是无法直接分块计算的。FlashAttention-2通过在线softmax解决这个问题：使用下标 i 标记当前查询区块，在query（行）维度上的小片大小是 $B_q$,同样地上标 j 来标记当前的键区块，键在key（行）维度的小片大小 $B_k$，而隐藏层维度d上保持连续存储，不进行分块。

我们要维护两组行级的中间变量--运行的行中最大值 $m_i^{(j)}\in\mathbb{R}^{B_q}$,以及分母的值 $l_i^{(j)}\in\mathbb{R}^{B_q}$，前者用于数值稳定(防止溢出的stable_softmax)，后者用于分母的运行代理；随着键区块索引 j 的递增，行中最大值逐次更新，并生成softmax分子项 $\tilde{P}_i^{(j)}=exp(S_{ij}-m_i^{(j)})$，同时代理值更新 $l_i^{(j)}=\exp(m_i^{(j-1)})$。最终输出归一化需使用所有键区块处理完成后的终值 $l_i^{(T_k)}$

在开始使用 Triton 编写分块的前向传播之前，得知道以下细节：

1.debug的时候可以使用tl.device_print打印语句，虽然提供TRITON-INTERPRET=1设置可以在CPU上运行Triton，但是会存在稳定性问题。

2.定义块指针时，务必确保偏移量正确，且区块偏移需要乘以相应的分块尺寸。

3.线程块的启动网格通过kernel_fn\[(launch_grid,launch_grid_d2,...)](块定义参数)实现，就像本Notebook之前所展示那样，该调用位于torch.autograd.Function子类中

4.矩阵乘法运算我们用tl.dot 来实现

5.推进块指针的位置的时候我们采用 *_block_ptr=*_block_ptr.advance(...) 的方法，advance后面就跟移动内存区域（元素）个数

In [4]:
import torch
import torch.autograd
import math
import torch.nn.functional as F
from einops import rearrange, einsum

In [7]:
#compile用法见pdf第九页
def flash_bwd(Q,K,V,O,dO,L,scale,is_causal=False):
        D=torch.sum(O*dO,dim=-1)
        S=torch.einsum("... q d,... k d -> ... q k",Q,K)
        S=S*scale
        if is_causal:
            n_queries = Q.shape[-2]
            n_keys = K.shape[-2]
            # 创建一个上三角掩码
            mask = torch.triu(torch.ones(n_queries, n_keys, device=Q.device, dtype=torch.bool), diagonal=1)
            S.masked_fill_(mask, -float('inf'))
        P=torch.exp(S-L.unsqueeze(-1))
        dV=torch.einsum("... q k, ... q d -> ... k d",P,dO)
        dP=torch.einsum("... q d, ... k d -> ... q k",dO,V)
        dS=P*(dP-D.unsqueeze(-1))
        dQ=torch.einsum("... q k, ... k d -> ... q d",dS,K)
        dQ=dQ*scale
        dK=torch.einsum("... q k, ... q d -> ... k d",dS,Q)
        #dK=torch.matmul(dS.transpose(-2,-1),Q)
        dK=dK*scale
        return dQ,dK,dV


compiled_bwd=torch.compile(flash_bwd)

class FlashAttention(torch.autograd.Function):
    @staticmethod
    def forward(ctx, Q,K,V,is_causal=False):
        original_Q_shape = Q.shape
        original_K_shape = K.shape
        original_V_shape = V.shape
        if len(Q.shape)==2:
            Q=Q.unsqueeze(0).unsqueeze(0)
            K=K.unsqueeze(0).unsqueeze(0)
            V=V.unsqueeze(0).unsqueeze(0)

        if len(K.shape)==3:
            Q=Q.unsqueeze(1)
            K=K.unsqueeze(1)
            V=V.unsqueeze(1)

        Q_shape_before,seq_len,d=Q.shape[:-2],Q.shape[-2],Q.shape[-1]
        B_q=min(16,seq_len)       #定义Q的块的行大小
        B_k=min(16,seq_len)       #定义KV的块的行大小，注意一件事K会进行转置，这个行分块转置后是裂分块

        T_q=math.ceil(seq_len/B_q)  #Q的分块数
        T_k=math.ceil(seq_len/B_k)  #K,V的分块数

        #对输出的O和维护的代理值l 进行空初始化
        O=torch.empty_like(Q)
        L=torch.empty((*Q_shape_before,seq_len),dtype=Q.dtype,device=Q.device)

        for i in range(T_q):
            q_start=i*B_q
            q_end=min((i+1)*B_q,seq_len)

            Q_i=Q[:,:,q_start:q_end,:]    #加载一个行分块
            O_i=torch.zeros(Q_i.shape,dtype=torch.float32,device=Q.device)     #小块的输出形状是(B,num_heads,B_q,d)
            l_i=torch.zeros((*Q_shape_before,q_end-q_start),dtype=torch.float32,device=Q.device)
            m_i=torch.full((*Q_shape_before,q_end-q_start),-float("inf"),dtype=torch.float32,device=Q.device)

            for j in range(T_k):
                kv_start=j*B_k
                kv_end=min((j+1)*B_k,seq_len)

                Kj=K[:,:,kv_start:kv_end,:]
                Vj=V[:,:,kv_start:kv_end,:]

                S_ij=torch.matmul(Q_i,Kj.transpose(-2,-1)/math.sqrt(d))

                m_ij=torch.max(S_ij,dim=-1)[0]   #维护这一行的最大值,形状成为(B,num_heads,B_q)
                m_i_new=torch.maximum(m_i,m_ij)

                P_ij=torch.exp(S_ij-m_i_new.unsqueeze(-1))           #前者(B,num_heads,B_q,B_k)，后者m_i_new变为(B,num_heads,B_q,1)，然后作广播减法，某一行全减去后者相同行的东西

                l_i_new=torch.exp(m_i-m_i_new)*l_i+torch.sum(P_ij,dim=-1) #形状(B,num_heads,B_q)
                scale_factor=torch.exp(m_i-m_i_new).unsqueeze(-1)   #算出exp自升一个维度 (B,num_heads,B_q,1)
                O_i=scale_factor*O_i +torch.matmul(P_ij,Vj)

                m_i=m_i_new
                l_i=l_i_new

            O_i=O_i/l_i.unsqueeze(-1)

            O[:,:,q_start:q_end,:]=O_i.to(Q.dtype)
            L[:,:,q_start:q_end]=m_i+torch.log(l_i)

        if len(original_Q_shape)==2:
            O=O.squeeze(0).squeeze(0)
            Q=Q.squeeze(0).squeeze(0)
            K=K.squeeze(0).squeeze(0)
            V=V.squeeze(0).squeeze(0)
            L=L.squeeze(0).squeeze(0)

        elif len(original_Q_shape)==3:
            O=O.squeeze(1)
            Q=Q.squeeze(1)
            K=K.squeeze(1)
            V=V.squeeze(1)
            L=L.squeeze(1)

        ctx.save_for_backward(L,Q,K,V,O)
        ctx.is_causal=is_causal
        return O

    @staticmethod
    def backward(ctx, grad_output):
        L,Q,K,V,O=ctx.saved_tensors
        d_model=Q.shape[-1]
        scale=1./math.sqrt(d_model)
        dQ,dK,dV=compiled_bwd(Q,K,V,O,grad_output,L,scale)

        return dQ, dK, dV,None

In [3]:
#刚才实施的那个纯torch版的可能还存在bug而且性能较差，下面编写triton核来并行实现。
import triton
import triton.language as tl

from einops import rearrange,einsum
import torch
import math

In [4]:
#刚才实施的那个纯torch版的可能还存在bug而且性能较差，下面编写triton核来并行实现。

@triton.jit
def flash_fwd_kernel(
        Q_ptr, K_ptr, V_ptr,  # 输入QKV矩阵存储区域的指针
        O_ptr, L_ptr,  # 输出O L存储区域的指针
        stride_qb, stride_qq, stride_qd,  # 分别是q移动一个批次跳过元素的步长，查询个数维度上移动一个单位跳过元素的步长，以及特征维度移动一次的步长(这个通常为1)
        stride_kb, stride_kk, stride_kd,  # K的同上
        stride_vb, stride_vk, stride_vd,  # V的同上
        stride_ob, stride_oq, stride_od,  # O的同上
        stride_lb, stride_lq,  # L仅有两个维度，原因是L是某行指数和的对数，所以自然没有d维度
        N_QUERIES, N_KEYS,  # 查询数和键数的总值
        scale,  # 点积缩放因子
        D: tl.constexpr,  # 特征的总维度
        Q_TILE_SIZE: tl.constexpr,  # 查询分块在query上的大小B_q
        K_TILE_SIZE: tl.constexpr,  # 键分块在Key上的大小B_k
        is_causal:tl.constexpr
):
    query_tile_index = tl.program_id(0)  # 获取该线程的查询区块索引
    batch_index = tl.program_id(1)  # 获取该线程的批次索引

    Q_block_ptr = tl.make_block_ptr(
        Q_ptr + batch_index * stride_qb,  # 经过批次索引找到当前批次的Q张量首指针
        shape=(N_QUERIES, D,),
        strides=(stride_qq, stride_qd,),
        offsets=(query_tile_index * Q_TILE_SIZE, 0,),
        block_shape=(Q_TILE_SIZE, D,),
        order=(1, 0),  # 这是默认序，事实上0是按行的轴，1是按列的轴
    )

    K_block_ptr = tl.make_block_ptr(
        K_ptr + batch_index * stride_kb,
        shape=(N_KEYS, D),
        strides=(stride_kk, stride_kd,),
        offsets=(0, 0),  # 也就是说在对Q分块进行矩阵乘法的时候，我们会重新遍历K
        block_shape=(K_TILE_SIZE, D,),
        order=(1, 0),
    )

    V_block_ptr = tl.make_block_ptr(
        V_ptr + batch_index * stride_vb,
        shape=(N_KEYS, D),
        strides=(stride_vk, stride_vd,),
        offsets=(0, 0),
        block_shape=(K_TILE_SIZE, D,),
        order=(1, 0),
    )

    O_block_ptr = tl.make_block_ptr(
        O_ptr + batch_index * stride_ob,
        shape=(N_QUERIES, D,),
        strides=(stride_oq, stride_od,),
        offsets=(query_tile_index * Q_TILE_SIZE, 0,),
        block_shape=(Q_TILE_SIZE, D,),
        order=(1, 0),
    )

    L_block_ptr = tl.make_block_ptr(
        L_ptr + batch_index * stride_lb,
        shape=(N_QUERIES,),
        strides=(stride_lq,),
        offsets=(query_tile_index * Q_TILE_SIZE,),
        block_shape=(Q_TILE_SIZE,),
        order=(0,),
    )

    Q_tile = tl.load(Q_block_ptr,boundary_check=(0,1),padding_option="zero")  # 利用指针加载到当前对应的Q小片,我们在主进程中会并行加载多个Q_tile，然后每个tile逐次在若干行上向右移动，小块形状(Q_TILE_SIZE,D,)
    # 然后开始实现算法
    # 有个问题，我们在纯torch内部是利用了empty来实现了内存优化，但是triton内部不用这样做，因为每次仅加载小块；不仅如此，我们也不能这么做，kernel是纯粹的计算单元，不具备动态分配或释放全局内存的能力，也就是说占用的内存在一开始分配区块的时候就定死了。另外计算单元需要用float32提高精度
    O_acc = tl.zeros((Q_TILE_SIZE, D,), dtype=tl.float32)

    l_acc = tl.zeros((Q_TILE_SIZE,), dtype=tl.float32)

    m_acc = tl.full((Q_TILE_SIZE,), -float("inf"), dtype=tl.float32)

    num_key_tiles = tl.cdiv(N_KEYS, K_TILE_SIZE)  # 算法中的T_k
    # 注意编写的时候我们没对Q的列再划分

    for j in range(num_key_tiles):

        K_tile = tl.load(K_block_ptr,boundary_check=(0,1),padding_option="zero")  # 小块形状(K_TILE_SIZE,D,)
        V_tile = tl.load(V_block_ptr,boundary_check=(0,1),padding_option="zero")  # 小块形状(K_TILE_SIZE,D,)

        # 计算注意力得分 S_ij=Q_i@K_j^T/sqrt(d)->(Q_TILE_SIZE,K_TILE_SIZE)
        S_tile=tl.zeros((Q_TILE_SIZE,K_TILE_SIZE), dtype=tl.float32)
        S_tile = tl.dot(Q_tile, tl.trans(K_tile),acc=S_tile) * scale

        # 这里看是否需要因果掩码
        if is_causal:
            query_offset = query_tile_index * Q_TILE_SIZE + tl.arange(0,Q_TILE_SIZE)  # 计算查询的绝对位置，在arange里面自动广播了Q_TILE_SIZE个，形成当前块的所有行标
            key_offset = j * K_TILE_SIZE + tl.arange(0, K_TILE_SIZE)  # 键的绝对位置
            # 这里做个解释，前者升维到[Q_TILE_SIZE,1]后者到[1,K_TILE_SIZE]，前者的每一行元素和后者的每一列元素进行判断，形成[Q_TILE,K_TILE]矩阵，只有前者的当前元素更大 才为1，否则为0，形成下三角
            causal_mask = query_offset[:, None] >= key_offset[None, :]
            # 应用掩码
            S_tile = tl.where(causal_mask, S_tile,S_tile-1e6)  # 0处设置为负无穷，这样在softmax层算出来就是0了

        if j == num_key_tiles - 1:  # 循环来到了最后一次,注意前面我们的计算用了向上取整，最后一次迭代是存在无效元的
            key_mask = tl.arange(0, K_TILE_SIZE) < (N_KEYS - j * K_TILE_SIZE)  # 无效位置被标记为0
            key_mask = key_mask[None, :]  # 自升一维[1,K_TILE_SIZE]便于广播
            key_mask = tl.broadcast_to(key_mask, [Q_TILE_SIZE, K_TILE_SIZE])  # 广播成[Q_TILE_SIZE,K_TILE_SIZE]
            S_tile = tl.where(key_mask, S_tile, -float("inf"))  # 1的位置还是S_tile,0的位置标记为负无穷，这样取e后就是0，不占权重


        # 更新维护行内最大值：m_i^j=max(m_i^{j-1},rowmax(S_i^j))
        m_new = tl.maximum(m_acc, tl.max(S_tile, axis=-1))  # m_new是广播求的最大值，形状是(Q_TILE_SIZE,)每个元素是每一行的当前最大值
        P_tile = tl.exp(S_tile - m_new[:, None])  # 把m_new自升维到(Q_TILE_SIZE,1),然后自动广播S_tile每行减去相同的max值

        # 计算新的代理值：l_i^j=exp(m_i^{j-1}-m_i^{j})*l_ij+rowsum(P_i^{j}) 注意*是逐元素乘法
        l_new = tl.exp(m_acc - m_new) * l_acc + tl.sum(P_tile,axis=-1)  # 总之就是传统softmax我们仅用tl.sum一部分就够了，但是这里为了数值稳定我们额外加了一个tl.exp *l_acc来作为分母值

        # 更新输出：O_i^j=diag(exp(m_i^{j-1}-m_i^{j}))@O_i^{j-1}+P_tile@V_tile
        scale_factor = tl.exp(m_acc - m_new)
        O_acc=O_acc * scale_factor[:, None]
        O_acc = tl.dot(P_tile.to(V_tile.dtype), V_tile,acc=O_acc)

        m_acc = m_new
        l_acc = l_new
        # 迭代过程中需要移动的只有K，V块指针
        K_block_ptr = K_block_ptr.advance((K_TILE_SIZE, 0))
        V_block_ptr = V_block_ptr.advance((K_TILE_SIZE, 0))

    O_final = O_acc / l_acc[:, None]  # 把l_acc变为[Q_TILE_SIZE,1]然后逐行相除
    # 计算Logsumexp L_i=m_i^{T_k}+log(l_i^{T_k})
    L_final = m_acc + tl.log(l_acc)

    O_final = O_final.to(O_block_ptr.type.element_ty)
    tl.store(O_block_ptr, O_final,boundary_check=(0,1))  # 存储指针，内容可由指针找到
    tl.store(L_block_ptr, L_final,boundary_check=(0,))

class FlashAttention2(torch.autograd.Function):
    @staticmethod
    def forward(ctx, Q, K, V, is_causal=False,scale=None):
        batch_size, n_queries, d_model = Q.shape
        _, n_keys, _ = K.shape

        if not scale:
            scale = 1.0 / math.sqrt(d_model)

        # 确保张量是连续存储
        Q = Q.contiguous()
        K = K.contiguous()
        V = V.contiguous()

        O = torch.empty_like(Q)  # 现在不是triton核，可用空初始化
        L = torch.empty((batch_size, n_queries,), device=Q.device, dtype=torch.float32)

        # 块规模不能超过对应维度总长
        Q_TILE_SIZE = min(16, n_queries)
        K_TILE_SIZE = min(16, n_keys)

        # Q的线程分割
        num_query_tiles = triton.cdiv(n_queries, Q_TILE_SIZE)

        # 设置triton核函数的处理网格，第一维我们先在某个批次上处理分割的线程，第二位我们处理完某个批次的线程后移动批次来处理下一个批次
        grid = (num_query_tiles, batch_size)

        flash_fwd_kernel[grid](
            Q, K, V, O, L,
            Q.stride(0), Q.stride(1), Q.stride(2),
            K.stride(0), K.stride(1), K.stride(2),
            V.stride(0), V.stride(1), V.stride(2),
            O.stride(0), O.stride(1), O.stride(2),
            L.stride(0), L.stride(1),
            n_queries, n_keys,
            scale,
            D=d_model,
            Q_TILE_SIZE=Q_TILE_SIZE, K_TILE_SIZE=K_TILE_SIZE,
            is_causal=is_causal
        )

        ctx.is_causal = is_causal
        ctx.save_for_backward(Q, K, V, O, L)
        ctx.scale = scale

        return O

    @staticmethod
    def backward(ctx, grad_output):
        Q, K, V, O, L = ctx.saved_tensors
        scale = ctx.scale
        is_causal = ctx.is_causal
        dQ, dK, dV = compiled_bwd(Q, K, V, O, grad_output, L, scale,is_causal)

        return dQ, dK, dV, None,None

#注意：目前的实现backward部分仍然是标准的直接载入，虽然采用了重计算，但是内存开销仍然不小，相对于forward的triton部分内存大的离谱。

In [5]:
#compile用法见pdf第九页
#这一段移到了note前面
# def flash_bwd(Q,K,V,O,dO,L,scale):
#         D=torch.sum(O*dO,dim=-1)
#         S=torch.einsum("... q d,... k d -> ... q k",Q,K)
#         S=S*scale
#         P=torch.exp(S-L.unsqueeze(-1))
#         dV=torch.einsum("... q k, ... q d -> ... k d",P,dO)
#         dP=torch.einsum("... q d, ... k d -> ... q k",dO,V)
#         dS=P*(dP-D.unsqueeze(-1))
#         dQ=torch.einsum("... q k, ... k d -> ... q d",dS,K)
#         dQ=dQ*scale
#         dK=torch.einsum("... q k, ... q d -> ... q d",dS,Q)
#         dK=dK*scale
#         return dQ,dK,dV

根据刚才的那个问题，我们想对backward部分进行进一步的性能优化，讲义上给出来的思路有

1.调整分块大小：事实上我们的gpu在面对2的整幂时候会拥有更好的性能，我们或许可以使用2幂的进一法

2.优化Triton的其它超参数

3.就是我们说的triton版反向传播

4.反向传播分次处理输入，第一次计算 $dQ$ 第二次计算 $dK,dV$ ，避免块间的原子操作同步

5.因果掩码的时候提前终止实例，跳过全零分块的计算

6.区分非掩码块和对角块，前者是可以直接计算的，而后者值需要单次比较

In [ ]:
import torch
import torch.autograd
import math
import torch.nn.functional as F

from einops import rearrange, einsum

import triton
import triton.language as tl
from sympy.abc import q
from torch.backends.cudnn import flags

# from FlashAttention import flash_bwd
# compiled_bwd=torch.compile(flash_bwd)

@triton.jit
def flash_fwd_kernel(
        Q_ptr, K_ptr, V_ptr,
        O_ptr, L_ptr,
        stride_qb, stride_qq, stride_qd,
        stride_kb, stride_kk, stride_kd,
        stride_vb, stride_vk, stride_vd,
        stride_ob, stride_oq, stride_od,
        stride_lb, stride_lq,
        N_QUERIES, N_KEYS,
        scale,
        D: tl.constexpr,
        Q_TILE_SIZE: tl.constexpr,
        K_TILE_SIZE: tl.constexpr,
        is_causal:tl.constexpr
):
    query_tile_index = tl.program_id(0)
    batch_index = tl.program_id(1)

    Q_block_ptr = tl.make_block_ptr(
        Q_ptr + batch_index * stride_qb,
        shape=(N_QUERIES, D,),
        strides=(stride_qq, stride_qd,),
        offsets=(query_tile_index * Q_TILE_SIZE, 0,),
        block_shape=(Q_TILE_SIZE, D,),
        order=(1, 0),
    )

    K_block_ptr = tl.make_block_ptr(
        K_ptr + batch_index * stride_kb,
        shape=(N_KEYS, D),
        strides=(stride_kk, stride_kd,),
        offsets=(0, 0),
        block_shape=(K_TILE_SIZE, D,),
        order=(1, 0),
    )

    V_block_ptr = tl.make_block_ptr(
        V_ptr + batch_index * stride_vb,
        shape=(N_KEYS, D),
        strides=(stride_vk, stride_vd,),
        offsets=(0, 0),
        block_shape=(K_TILE_SIZE, D,),
        order=(1, 0),
    )

    O_block_ptr = tl.make_block_ptr(
        O_ptr + batch_index * stride_ob,
        shape=(N_QUERIES, D,),
        strides=(stride_oq, stride_od,),
        offsets=(query_tile_index * Q_TILE_SIZE, 0,),
        block_shape=(Q_TILE_SIZE, D,),
        order=(1, 0),
    )

    L_block_ptr = tl.make_block_ptr(
        L_ptr + batch_index * stride_lb,
        shape=(N_QUERIES,),
        strides=(stride_lq,),
        offsets=(query_tile_index * Q_TILE_SIZE,),
        block_shape=(Q_TILE_SIZE,),
        order=(0,),
    )

    Q_tile = tl.load(Q_block_ptr,boundary_check=(0,1),padding_option="zero")
    O_acc = tl.zeros((Q_TILE_SIZE, D,), dtype=tl.float32)
    l_acc = tl.zeros((Q_TILE_SIZE,), dtype=tl.float32)
    m_acc = tl.full((Q_TILE_SIZE,), -float("inf"), dtype=tl.float32)
    num_key_tiles = tl.cdiv(N_KEYS, K_TILE_SIZE)

    for j in range(num_key_tiles):
        flag=True        #flag来作为一个是否运行后面代码的指标
        if is_causal:
            query_end_pos=(query_tile_index+1)*Q_TILE_SIZE    #Q块完了之后的最后一行的下一行偏移
            key_start_pos=j*K_TILE_SIZE                       #K小块的起始偏移
            #如果Q块的结束位置下一个 是比K块起始偏移小（或等），那么就总有 横标小于纵标，从而整个块都可以定义为一个掩码值
            if key_start_pos>=query_end_pos:
                flag=False

        if flag:

            K_tile = tl.load(K_block_ptr, boundary_check=(0, 1), padding_option="zero")
            V_tile = tl.load(V_block_ptr,boundary_check=(0,1),padding_option="zero")

            S_tile=tl.zeros((Q_TILE_SIZE,K_TILE_SIZE), dtype=tl.float32)
            S_tile = tl.dot(Q_tile, tl.trans(K_tile),acc=S_tile) * scale

            #思考：1.仅对角处需要掩码的计算，对角线上方的可以直接跳过，而对角线下方直接计算非掩码即可；2.用了padding_option时，我们无需再去验证最后的一次j是否存在空值
            #第一个优化是无法在这里的下面实现的，因为triton内部在进入分支过后又会串行进行处理，而它的实现只能在一开始的时候就去判断,我们利用一个示性的flag来实现
            if is_causal:
                query_offset = query_tile_index * Q_TILE_SIZE + tl.arange(0,Q_TILE_SIZE)
                key_offset = j * K_TILE_SIZE + tl.arange(0, K_TILE_SIZE)
                causal_mask = query_offset[:, None] >= key_offset[None, :]
                S_tile = tl.where(causal_mask, S_tile,S_tile-1e6)

            m_new = tl.maximum(m_acc, tl.max(S_tile, axis=-1))
            P_tile = tl.exp(S_tile - m_new[:, None])

            l_new = tl.exp(m_acc - m_new) * l_acc + tl.sum(P_tile,axis=-1)

            scale_factor = tl.exp(m_acc - m_new)
            O_acc=O_acc * scale_factor[:, None]
            O_acc = tl.dot(P_tile.to(V_tile.dtype), V_tile,acc=O_acc)

            m_acc = m_new
            l_acc = l_new



        K_block_ptr = K_block_ptr.advance((K_TILE_SIZE, 0))
        V_block_ptr = V_block_ptr.advance((K_TILE_SIZE, 0))

    O_final = O_acc / l_acc[:, None]
    L_final = m_acc + tl.log(l_acc)

    O_final = O_final.to(O_block_ptr.type.element_ty)
    tl.store(O_block_ptr, O_final,boundary_check=(0,1))
    tl.store(L_block_ptr, L_final,boundary_check=(0,))

@triton.jit
def bwd_calculate_d_kernel(
        O_ptr,dO_ptr,
        D_ptr,
        stride_ob,stride_oq,stride_od,
        stride_dob,stride_doq,stride_dod,
        stride_db,stride_dq,
        N_QUERIES,
        D_MODEL:tl.constexpr,
        BLOCK_SIZE:tl.constexpr
):
    batch_idx=tl.program_id(0)
    query_tile_idx=tl.program_id(1)

    O_block_ptr=tl.make_block_ptr(
        O_ptr+batch_idx*stride_ob,
        shape=(N_QUERIES,D_MODEL,),
        strides=(stride_oq,stride_od,),
        offsets=(query_tile_idx*BLOCK_SIZE,0,),
        block_shape=(BLOCK_SIZE,D_MODEL,),
        order=(1,0),
    )
    dO_block_ptr=tl.make_block_ptr(
        dO_ptr+batch_idx*stride_dob,
        shape=(N_QUERIES,D_MODEL,),
        strides=(stride_doq,stride_dod,),
        offsets=(query_tile_idx*BLOCK_SIZE,0,),
        block_shape=(BLOCK_SIZE,D_MODEL,),
        order=(1,0),
    )
    O_row=tl.load(O_block_ptr,boundary_check=(0,),padding_option="zero")
    dO_row=tl.load(dO_block_ptr,boundary_check=(0,),padding_option="zero")

    D_row=tl.sum(O_row.to(tl.float32)*dO_row.to(tl.float32),axis=-1)    #形成形状为(BLOCK_SIZE,)的张量
    D_block_ptr=tl.make_block_ptr(
        D_ptr+batch_idx*stride_db,
        shape=(N_QUERIES,),
        strides=(stride_dq,),
        offsets=(query_tile_idx*BLOCK_SIZE,),
        block_shape=(BLOCK_SIZE,),
        order=(0,)
    )
    tl.store(D_block_ptr,D_row,boundary_check=(0,))

@triton.jit
def flash_bwd_kernel(
        Q_ptr,K_ptr,V_ptr
        #O_ptr
        ,L_ptr,dO_ptr,D_ptr,dQ_ptr,dK_ptr,dV_ptr,
        stride_qb,stride_qq,stride_qd,
        stride_kb,stride_kk,stride_kd,
        stride_vb,stride_vk,stride_vd,
        #stride_ob,stride_oq,stride_od,
        stride_lb,stride_lq,
        stride_dob,stride_doq,stride_dod,
        stride_db,stride_dq,
        stride_dqb,stride_dqq,stride_dqd,
        stride_dkb,stride_dkk,stride_dkd,
        stride_dvb,stride_dvk,stride_dvd,
        N_QUERIES,N_KEYS,
        scale,
        D:tl.constexpr,                                       #注意这个D是dim，而非计算的中间值D
        Q_TILE_SIZE:tl.constexpr,
        K_TILE_SIZE:tl.constexpr,
        is_causal:tl.constexpr
):
    key_tile_idx=tl.program_id(0)                             #注意反向传播的算法是外层为K，V循环，但是在这里我们会把循环调成并行
    batch_idx=tl.program_id(1)                                #先并行把K V的完成后 再调整批次

    K_block_ptr=tl.make_block_ptr(
        K_ptr+batch_idx*stride_kb,
        shape=(N_KEYS,D,),
        strides=(stride_kk,stride_kd,),
        offsets=(key_tile_idx*K_TILE_SIZE,0,),
        block_shape=(K_TILE_SIZE,D,),
        order=(1,0)
    )

    V_blcok_ptr=tl.make_block_ptr(
        V_ptr+batch_idx*stride_vb,
        shape=(N_KEYS,D,),
        strides=(stride_vk,stride_vd,),
        offsets=(key_tile_idx*K_TILE_SIZE,0,),
        block_shape=(K_TILE_SIZE,D,),
        order=(1,0)
    )

    K_tile=tl.load(K_block_ptr,boundary_check=(0,1),padding_option="zero")
    V_tile=tl.load(V_blcok_ptr,boundary_check=(0,1),padding_option="zero")

    dK_acc=tl.zeros((K_TILE_SIZE,D,),dtype=tl.float32)
    dV_acc=tl.zeros((K_TILE_SIZE,D,),dtype=tl.float32)

    Q_block_ptr=tl.make_block_ptr(
        Q_ptr+batch_idx*stride_qb,
        shape=(N_QUERIES,D,),
        strides=(stride_qq,stride_qd,),
        offsets=(0,0),                                 #每次都把偏移搞到矩阵左上角，我们每个线程是K V的固定整行，然后去遍历Q块，内层会进行advance，所以只需要在每个线程内至于原点即可
        block_shape=(Q_TILE_SIZE,D,),
        order=(1,0)
    )

    # O_block_ptr=tl.make_block_ptr(
    #     O_ptr+batch_idx*stride_ob,
    #     shape=(N_QUERIES,D,),
    #     strides=(stride_oq,stride_od,),
    #     offsets=(0,0),
    #     block_shape=(Q_TILE_SIZE,D,),
    #     order=(1,0)
    # )

    dO_block_ptr=tl.make_block_ptr(
        dO_ptr+batch_idx*stride_dob,
        shape=(N_QUERIES,D,),
        strides=(stride_doq,stride_dod,),
        offsets=(0,0),
        block_shape=(Q_TILE_SIZE,D,),
        order=(1,0)
    )

    # dQ_block_ptr=tl.make_block_ptr(
    #     dQ_ptr+batch_idx*stride_dqb,                之前设想的像forward一样的flag实现貌似有问题，我们现在正常掩码
    #     shape=(N_QUERIES,D,),
    #     strides=(stride_dqq,stride_dqd,),
    #     offsets=(0,0),
    #     block_shape=(Q_TILE_SIZE,D,),
    #     order=(1,0)
    # )

    L_block_ptr=tl.make_block_ptr(
        L_ptr+batch_idx*stride_lb,
        shape=(N_QUERIES,),
        strides=(stride_lq,),
        offsets=(0,),
        block_shape=(Q_TILE_SIZE,),
        order=(0,)
    )
    D_block_ptr=tl.make_block_ptr(
        D_ptr+batch_idx*stride_db,
        shape=(N_QUERIES,),
        strides=(stride_dq,),
        offsets=(0,),
        block_shape=(Q_TILE_SIZE,),
        order=(0,)
    )


    num_query_tiles=tl.cdiv(N_QUERIES,Q_TILE_SIZE)      #这里内层循环是Q

    for i in range(num_query_tiles):
        Q_tile=tl.load(Q_block_ptr,boundary_check=(0,1),padding_option="zero")

        dO_tile=tl.load(dO_block_ptr,boundary_check=(0,1),padding_option="zero")
        L_tile=tl.load(L_block_ptr,boundary_check=(0,),padding_option="zero")
        D_tile=tl.load(D_block_ptr,boundary_check=(0,),padding_option="zero")

        S_tile = tl.dot(Q_tile, tl.trans(K_tile)) * scale
        P_tile = tl.exp(S_tile - L_tile[:, None])
        # dV_acc =dV_acc+tl.dot(tl.trans(P_tile), dO_tile.to(tl.float32))
        # dP_tile = tl.dot(dO_tile.to(tl.float32), tl.trans(V_tile).to(tl.float32))
        # dS_tile=P_tile.to(tl.float32)*(dP_tile-D_tile[:,None])

        if is_causal:
            query_offset=i*Q_TILE_SIZE+tl.arange(0,Q_TILE_SIZE)           #把掩码码处的值设为0来使得梯度也清空
            key_offset=key_tile_idx*K_TILE_SIZE+tl.arange(0,K_TILE_SIZE)
            causal_mask=query_offset[:,None]>=key_offset[None,:]
            P_tile=tl.where(causal_mask,P_tile,0.0)
            #dS_tile=tl.where(causal_mask,dS_tile,0.0)

        dV_acc = dV_acc + tl.dot(tl.trans(P_tile), dO_tile.to(tl.float32))
        dP_tile = tl.dot(dO_tile.to(tl.float32), tl.trans(V_tile).to(tl.float32))
        dS_tile = P_tile.to(tl.float32) * (dP_tile - D_tile[:, None])


        dS_tile_f32=dS_tile.to(tl.float32)
        dQ_i=tl.dot(dS_tile_f32,K_tile.to(tl.float32))*scale            #中间的加值

        dK_acc=dK_acc+tl.dot(tl.trans(dS_tile_f32),Q_tile.to(tl.float32))*scale

        query_offsets=i*Q_TILE_SIZE+tl.arange(0,Q_TILE_SIZE)
        dim_offsets=tl.arange(0,D)
        dQ_tile_ptr=dQ_ptr+batch_idx*stride_dqb+query_offsets[:,None]*stride_dqq+dim_offsets[None,:]*stride_dqd
        query_mask=query_offsets<N_QUERIES
        tl.atomic_add(dQ_tile_ptr,dQ_i,mask=query_mask[:,None])

        Q_block_ptr=Q_block_ptr.advance((Q_TILE_SIZE,0))
        dO_block_ptr=dO_block_ptr.advance((Q_TILE_SIZE,0))

        L_block_ptr=L_block_ptr.advance((Q_TILE_SIZE,))
        D_block_ptr=D_block_ptr.advance((Q_TILE_SIZE,))
    dK_block_ptr=tl.make_block_ptr(
        dK_ptr+batch_idx*stride_dkb,
        shape=(N_KEYS,D,),
        strides=(stride_dkk,stride_dkd,),
        offsets=(key_tile_idx*K_TILE_SIZE,0,),
        block_shape=(K_TILE_SIZE,D,),
        order=(1,0)
    )
    dV_block_ptr=tl.make_block_ptr(
        dV_ptr+batch_idx*stride_dvb,
        shape=(N_KEYS,D,),
        strides=(stride_dvk,stride_dvd,),
        offsets=(key_tile_idx*K_TILE_SIZE,0,),
        block_shape=(K_TILE_SIZE,D,),
        order=(1,0)
    )

    tl.store(dK_block_ptr,dK_acc.to(dK_block_ptr.type.element_ty),boundary_check=(0,1))
    tl.store(dV_block_ptr,dV_acc.to(dV_block_ptr.type.element_ty),boundary_check=(0,1))




class FlashAttention2(torch.autograd.Function):
    @staticmethod
    def forward(ctx, Q, K, V, is_causal=False,scale=None):
        #思考：如下实现中仅考虑了3个维度的形式，对于增加了头维度的，我们记录头的维度，利用einops来变换为3个维度的；
        #另一方面，triton核是在每个线程上按列进行的，而此时处理2的幂次的张量效果更为显著，我们的实现中d上是一次性处理完的，我们对d进行2的幂次进一 o90
        original_shape=Q.shape
        d_model=original_shape[-1]
        if not scale:
            scale = 1.0 / math.sqrt(d_model)
        #2幂的进一
        next_pow_2_d=triton.next_power_of_2(d_model)
        flag_padding=(next_pow_2_d!=d_model)

        Q=rearrange(Q,"... n d -> (...) n d")
        K=rearrange(K,"... n d -> (...) n d")
        V=rearrange(V,"... n d -> (...) n d")

        if flag_padding:
            pad_width=next_pow_2_d-d_model
            Q=F.pad(Q,(0,pad_width))
            K=F.pad(K,(0,pad_width))
            V=F.pad(V,(0,pad_width))

        Q = Q.contiguous()
        K = K.contiguous()
        V = V.contiguous()

        new_batch_size,n_queries,_=Q.shape
        _,n_keys,_=K.shape

        if flag_padding:
            O = torch.empty((new_batch_size,n_queries,d_model),device=Q.device,dtype=Q.dtype)
        else:
            O=torch.empty_like(Q)

        L = torch.empty((new_batch_size, n_queries,), device=Q.device, dtype=torch.float32)

        Q_TILE_SIZE = min(16, n_queries)
        K_TILE_SIZE = min(16, n_keys)

        num_query_tiles = triton.cdiv(n_queries, Q_TILE_SIZE)

        grid = (num_query_tiles, new_batch_size)

        flash_fwd_kernel[grid](
            Q, K, V, O, L,
            Q.stride(0), Q.stride(1), Q.stride(2),
            K.stride(0), K.stride(1), K.stride(2),
            V.stride(0), V.stride(1), V.stride(2),
            O.stride(0), O.stride(1), O.stride(2),
            L.stride(0), L.stride(1),
            n_queries, n_keys,
            scale,
            D=next_pow_2_d,
            Q_TILE_SIZE=Q_TILE_SIZE, K_TILE_SIZE=K_TILE_SIZE,
            is_causal=is_causal
        )
        #还原
        if flag_padding:
            O=O[...,:d_model]

        #使用original_shape来恢复
        O=O.reshape(*original_shape[:-2],*O.shape[1:])

        assert O.shape==original_shape
        ctx.is_causal = is_causal
        ctx.save_for_backward(Q, K, V, O, L)
        ctx.scale = scale
        ctx.flag_padding=flag_padding
        ctx.next_pow_2_d=next_pow_2_d

        return O

    @staticmethod
    def backward(ctx, grad_output):
        Q, K, V, O, L = ctx.saved_tensors
        scale = ctx.scale
        is_causal = ctx.is_causal
        flag_padding = ctx.flag_padding
        next_pow_2_d=ctx.next_pow_2_d

        dO=grad_output.contiguous()
        d_model=Q.shape[-1]

        dO_3d = rearrange(dO, "... n d -> (...) n d")
        Q_3d=rearrange(Q,"... n d -> (...) n d")
        K_3d=rearrange(K,"... n d -> (...) n d")
        V_3d=rearrange(V,"... n d -> (...) n d")
        O_3d=rearrange(O,"... n d -> (...) n d")


        if flag_padding:
            pad_width=next_pow_2_d-d_model
            dO_3d=F.pad(dO_3d,(0,pad_width))
            Q_3d=F.pad(Q_3d,(0,pad_width))
            K_3d=F.pad(K_3d,(0,pad_width))
            V_3d=F.pad(V_3d,(0,pad_width))
            O_3d=F.pad(O_3d,(0,pad_width))

        grid_batch_size,n_queries,_=Q_3d.shape
        _,n_keys,_=K_3d.shape

        D=torch.empty_like(L)

        D_BLOCK_SIZE=min(16,n_queries)
        d_grid=(grid_batch_size,triton.cdiv(n_queries,D_BLOCK_SIZE))       #为计算D来并行

        bwd_calculate_d_kernel[d_grid](
            O_3d,dO_3d,D,
            O_3d.stride(0),O_3d.stride(1),O_3d.stride(2),
            dO_3d.stride(0),dO_3d.stride(1),dO_3d.stride(2),
            D.stride(0),D.stride(1),
            n_queries,D_MODEL=next_pow_2_d,BLOCK_SIZE=D_BLOCK_SIZE
        )

        dQ_3d=torch.zeros_like(Q_3d,dtype=torch.float32)
        dK_3d=torch.zeros_like(K_3d,dtype=torch.float32)
        dV_3d=torch.zeros_like(V_3d,dtype=torch.float32)

        Q_TILE_SIZE = min(16, n_queries)
        K_TILE_SIZE = min(16, n_keys)

        bwd_grid=(triton.cdiv(n_keys,K_TILE_SIZE),grid_batch_size)

        flash_bwd_kernel[bwd_grid](
            Q_3d,K_3d,V_3d,L,dO_3d,D,dQ_3d,dK_3d,dV_3d,
            Q_3d.stride(0), Q_3d.stride(1), Q_3d.stride(2),
            K_3d.stride(0), K_3d.stride(1), K_3d.stride(2),
            V_3d.stride(0), V_3d.stride(1), V_3d.stride(2),
            L.stride(0), L.stride(1),
            dO_3d.stride(0), dO_3d.stride(1), dO_3d.stride(2),
            D.stride(0), D.stride(1),
            dQ_3d.stride(0), dQ_3d.stride(1), dQ_3d.stride(2),
            dK_3d.stride(0), dK_3d.stride(1), dK_3d.stride(2),
            dV_3d.stride(0), dV_3d.stride(1), dV_3d.stride(2),
            n_queries,n_keys,scale,D=next_pow_2_d,Q_TILE_SIZE=Q_TILE_SIZE,K_TILE_SIZE=K_TILE_SIZE,is_causal=is_causal,
        )
        if flag_padding:
            dQ_3d=dQ_3d[...,:d_model]
            dK_3d=dK_3d[...,:d_model]
            dV_3d=dV_3d[...,:d_model]

        dQ=dQ_3d.reshape(*Q.shape[:-2],*dQ_3d.shape[1:]).to(Q.dtype)
        dK=dK_3d.reshape(*K.shape[:-2],*dK_3d.shape[1:]).to(K.dtype)
        dV=dV_3d.reshape(*V.shape[:-2],*dV_3d.shape[1:]).to(V.dtype)
        return dQ,dK,dV,None,None